# Predicting Behavioral Outcomes from Neural Spike Train Data: An Analysis of Visual Stimulus Experiments

#### STA 141 Final Project  by Yingchun Yuan 919552555

---

## **Abstract**

This project examines the use of neural spike train data to predict the outcomes of visual stimulus experiments in mice. The analysis is based on data collected from 18 sessions involving four mice. Our core question is how to predict trial outcomes from these spike trains, given the complexity of high-dimensional neuron data spanning multiple brain areas.

---

## **Session 1: Introduction**

The focus of this project is to develop a predictive model that uses neural spike train data and visual stimulus contrast information to forecast behavioral outcomes in mice. 

The dataset, originating from experiments on four mice over 18 sessions, includes detailed records of trial feedback, stimulus contrasts, and time-resolved spike counts from various brain areas. An initial exploratory analysis identifies important data features and potential patterns, while a data integration step consolidates trial-level information across sessions. Finally, a logistic regression model is applied to predict trial outcomes, taking into account both stimulus conditions and the summarized neural activity. This approach aims to provide a clear demonstration of effective data processing and predictive modeling techniques in a practical project setting.

This project examines neural activity data from 18 sessions collected from four mice—Cori, Forssmann, Hench, and Lederberg. Each session includes the following important variables:

- **Contrast**: The contrast levels of the stimulus presented on either the left or right side. This determines the sensory input the mice receives.
- **Feedback**: Indicates whether the trial was successful (1) or a failure (-1), providing a measure of trial performance.
- **Time**: Represents the time bin centers used to segment neural spike activity.
- **Spikes**: The recorded spike counts of neurons within the visual cortex, measured across predefined time bins.
- **Brain Area**: Identifies the specific region of the brain where each recorded neuron is located, allowing for spatial analysis of neural activity.

We analyze data integration steps that account for session-to-session differences and highlight the significance of contrast-based stimuli in shaping behavior.

By focusing on shared features, we aim to build a model that generalizes well across different mice and experimental sessions, ultimately connecting neural signatures to decision-making processes.

---

**## Session 2: Exploratory Analysis**

#### **Overview of Sessions and Neurons**

|mouse_name |date_exp   | n_brain_area| n_neurons| n_trials| success_rate|
|:----------|:----------|------------:|---------:|--------:|------------:|
|Cori       |2016-12-14 |            8|       734|      114|    0.6052632|
|Cori       |2016-12-17 |            5|      1070|      251|    0.6334661|
|Cori       |2016-12-18 |           11|       619|      228|    0.6622807|
|Forssmann  |2017-11-01 |           11|      1769|      249|    0.6666667|
|Forssmann  |2017-11-02 |           10|      1077|      254|    0.6614173|
|Forssmann  |2017-11-04 |            5|      1169|      290|    0.7413793|
|Forssmann  |2017-11-05 |            8|       584|      252|    0.6706349|
|Hench      |2017-06-15 |           15|      1157|      250|    0.6440000|
|Hench      |2017-06-16 |           12|       788|      372|    0.6854839|
|Hench      |2017-06-17 |           13|      1172|      447|    0.6196868|
|Hench      |2017-06-18 |            6|       857|      342|    0.7953216|
|Lederberg  |2017-12-05 |           12|       698|      340|    0.7382353|
|Lederberg  |2017-12-06 |           15|       983|      300|    0.7966667|
|Lederberg  |2017-12-07 |           10|       756|      268|    0.6940299|
|Lederberg  |2017-12-08 |            8|       743|      404|    0.7648515|
|Lederberg  |2017-12-09 |            6|       474|      280|    0.7178571|
|Lederberg  |2017-12-10 |            6|       565|      224|    0.8303571|
|Lederberg  |2017-12-11 |           10|      1090|      216|    0.8055556|
<center>Table 1: Data Overview</center>

Our exploratory analysis reveals considerable variability across sessions. We compiled a metadata table summarizing each session's mouse identity, experiment date, number of brain areas, number of neurons, number of trials, and success rate. For example, the metadata table shows that session 1 (Cori, 2016-12-14) recorded 734 neurons over 114 trials with a success rate of 60.5%, whereas other sessions show significant differences in these metrics.

#### **Distribution of Contrast Differences**

<center><img src="./Graphs/ContrastDifference.png" alt="Distribution of Contrast Difference" style="width: 400px;" align="center"/></center>
<center>Figure 1: Contrast Difference Distribution</center>

By computing the absolute difference between the left and right stimulus contrasts, we obtained a contrast difference variable. The figure shows that most trials have a contrast difference ranging from 0 to 1, with a peak around 0.0，suggesting that many trials involve moderate differences in stimulus intensity.

#### **Success Rate Analysis**

<center><img src="./Graphs/Success Rate Analysis.png" alt="Success Rate Analysis" style="width: 400px;" align="center"/></center>
<center>Figure 2: Relationship between Contrast Difference Bin and Success Rate</center>

We further examined how success rates vary across different contrast difference bins. The bar chart demonstrates that higher contrast differences are generally associated with higher success rates, confirming that the trial becomes easier to success as the stimulus disparity increases.

#### **Mouse-Specific Performance**

<center><img src="./Graphs/Mouse-Specific Performance.png" alt="Mouse-Specific Performance" style="width: 400px;" align="center"/></center>
<center>Figure 3: Relationship between mice species and success rate</center>

Additionally, we analyzed success rates by mouse. The results, displayed in the figure, indicate distinct performance patterns among the four mice, with Lederberg tending to have higher success rates. This individual variability suggests that factors such as visual discrimination ability and engagement may differ across subjects.

---

## **Session 3: Data Integration**

#### **Standardizing Neural Features**

```R
all_brain_areas <- unique(unlist(lapply(session, function(s) unique(s$brain_area))))
all_brain_areas <- sort(all_brain_areas)
print(all_brain_areas)
```

```R
integrated_data <- tibble()
for(s in 1:18){
  n.trials <- length(session[[s]]$feedback_type)
  tmp <- session[[s]]
  brain_areas_session <- tmp$brain_area
  for(trial in 1:n.trials){
    spks_trial <- tmp$spks[[trial]]
    total_spikes <- apply(spks_trial, 1, sum)
    brain_area_features <- setNames(rep(0, length(all_brain_areas)), paste0("brain_area_", all_brain_areas))
    for(area in unique(brain_areas_session)){
      idx <- which(brain_areas_session == area)
      if(length(idx) > 0){
         brain_area_features[paste0("brain_area_", area)] <- mean(total_spikes[idx])
      }
    }
    trial_info <- tibble(
      session_ID = s,
      trial_ID = trial,
      mouse_name = tmp$mouse_name,
      contrast_left = tmp$contrast_left[trial],
      contrast_right = tmp$contrast_right[trial],
      feedback_type = tmp$feedback_type[trial],
      avg_spks = tmp$avg.spks[trial]
    )
    trial_features <- as_tibble(as.list(brain_area_features))
    trial_combined <- bind_cols(trial_info, trial_features)
    integrated_data <- bind_rows(integrated_data, trial_combined)
  }
}
head(integrated_data)
```

To address the heterogeneity across sessions, we integrated trial-level data by extracting brain area features. First, we identified all unique brain areas across sessions. For each trial, we computed the average spike count per brain area and combined these features with trial metadata. This resulted in an integrated dataset that contains both the original trial features (contrast levels, feedback, etc.) and a comprehensive set of brain area features.

#### **Additional Feature Engineering**

We further enhanced the integrated dataset by adding new features such as the contrast difference and a decision variable. The decision variable categorizes trials based on whether the left or right contrast was dominant, or if there was no stimulus. These additional features enrich the dataset and are expected to improve the predictive model’s performance.

---

## **Session 4: Predictive Modeling**

#### **Model Development**

We developed a logistic regression model using our integrated features—including stimulus contrasts (contrast_left and contrast_right), average spike count (avg_spks), contrast difference (contrast_diff), decision variable (decision), session identifier (session_ID), and a suite of brain area features. This model was chosen for its interpretability and its ability to handle the high-dimensional, heterogeneous neural data collected across 18 sessions.

############## TEST DATA SET
- **contrast_right**: Estimate = -1.249599, p < 0.001, indicating that higher right stimulus contrast is associated with lower odds of success.
- **avg_spks**: Estimate = 3.053185, p < 0.001, suggesting that increased overall neural activity significantly increases the likelihood of success.
- **contrast_diff**: Estimate = 1.260918, p < 0.001, which implies that larger contrast differences (i.e., greater stimulus disparity) are linked to higher trial success.

These coefficients suggest that both stimulus parameters and neural activity patterns—particularly in specific brain regions (e.g., brain_area_CA1, brain_area_BLA, brain_area_LD, brain_area_GPe, and brain_area_RSP)—play critical roles in predicting behavioral outcomes.

#### **Model Interpretation**

The logistic regression model, with an AIC of 4540.9, indicates a reasonable fit given the complexity of the data. Notably, the positive association of avg_spks and contrast_diff with trial success underscores the importance of neural firing rates and stimulus disparity in driving behavior. Conversely, the negative coefficient for contrast_right may reflect a lateralized effect or specific task demands.

---

## **Session 5: Prediction Performance on the Test Sets**

#### **Evaluation Metrics**

<center><img src="./Graphs/ROC Curve.png" alt="ROC Curve" style="width: 400px;" align="center"/></center>
<center>Figure 4: Receiver Operating Characteristic figure of the Data</center>

**<center>Test Accuracy: 0.71779744346116**</center>
**<center>**AUC: 0.667435847399747**</center>

We assessed the performance of our final logistic regression model on a held-out test set. The model achieved an overall accuracy of approximately **71.78%**. In addition, the ROC curve analysis yielded an AUC of **0.667**, indicating moderate discriminative ability between successful and unsuccessful trials.

#### **Confusion Matrix Analysis**

The confusion matrix reveals the following key statistics:
- **True Negatives (TN)**: 44
- **False Positives (FP)**: 54
- **False Negatives (FN)**: 233
- **True Positives (TP)**: 686

From these results, the model exhibits a high specificity of **92.7%** but a low sensitivity of **15.9%**. This suggests that while the model is effective at correctly identifying failure trials, it tends to under-predict success, likely reflecting the imbalance in the dataset. Additionally, the positive predictive value for predicting failure is **44.9%**, and the overall balanced accuracy is **54.3%**.

#### **Overall Performance**

Overall, while the logistic regression model demonstrates an acceptable level of accuracy, the observed bias towards predicting success indicates that further improvements are needed. Potential strategies for future work include incorporating class-balancing techniques, exploring more advanced ensemble methods, or employing alternative modeling approaches such as gradient boosting to better capture the complex relationships inherent in the neural data.

---

## **Session 6: Discussion**

In this project, we investigated the use of neural spike train data from multiple sessions to predict the behavioral outcomes of visual discrimination tasks in mice. Our approach involved an initial exploratory analysis to characterize the dataset, followed by a data integration process that consolidated trial-level features—including stimulus contrasts, neural firing rates, and brain area-specific activity—across 18 sessions.

#### **Key Findings**

- **Stimulus and Neural Activity Contributions**: Our logistic regression model indicated that variables such as contrast_right, avg_spks, and contrast_diff were statistically significant predictors of trial outcomes. In particular, a higher contrast difference and increased neural firing rates were associated with a greater likelihood of success, suggesting that both the sensory input and overall neural activity contribute meaningfully to decision-making.
- **Brain Region Effects**: Several brain area features (e.g., brain_area_CA1, brain_area_BLA, brain_area_LD, brain_area_GPe, and brain_area_RSP) emerged as important factors in the predictive model. This highlights the potential role of specific neural circuits in mediating the mice’s performance on visual tasks.
- **Model Performance and Limitations**: Although our model achieved an overall test accuracy of approximately 71.8% and an AUC of 0.667, the confusion matrix revealed a pronounced imbalance: the model exhibited high specificity but low sensitivity. This indicates that while the model reliably identifies failure trials, it tends to under-predict successful trials. Such an imbalance might be due to class imbalance in the data or inherent variability across sessions.

#### **Limitations**

Waiting for test dataset